In [9]:
import pandas as pd
import yfinance as yf

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.main import check_and_make_directories
from pprint import pprint
from stable_baselines3.common.logger import configure

from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

from finrl.config_tickers import DOW_30_TICKER

import talib

Создаем служебные каталоги

In [3]:
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

Загружаем исходные данные

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2019-01-01'
TEST_START_DATE = '2019-01-01'
TEST_END_DATE = '2021-01-01'

# загружаем данные
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (88061, 8)


Преобразуем дату и сортируем датасет по дате и бумагам

In [5]:
# преобразуем дату
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

# Отсортируем данные по дате и коду акции
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.740172,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,42.107315,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.098143,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941097,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,30.837599,7117200,CAT,4


Выберем только бумаги Apple

In [8]:
data = df.loc[df['tic'] == 'AAPL']
data.drop(['tic', 'day'], axis=1, inplace=True)
data.head()

,date,open,high,low,close,volume
0,2009-01-02,3.067143,3.251429,3.041429,2.740172,746015200
29,2009-01-05,3.327500,3.435000,3.311071,2.855818,1181608400
58,2009-01-06,3.426786,3.470357,3.299643,2.808715,1289310400
87,2009-01-07,3.278929,3.303571,3.223571,2.748024,753048800
116,2009-01-08,3.229643,3.326786,3.215714,2.799052,673500800


## Технический анализ

Будем формировать показатели с использованием библиотеки TA-Lib (https://ta-lib.org)

### Простое скользящее среднее (Simple Moving Averages - SMA)

Скользящие средние - один из самых популярных технических индикаторов, используемых для сглаживания ценовых колебаний и выявления трендов. Они могут рассчитываться с использованием различных временных периодов, таких как краткосрочный (10 дней), среднесрочный (50 дней) и долгосрочный (200 дней).

In [11]:
data["sma_10"] = talib.SMA(data["close"], timeperiod=10)
data["sma_50"] = talib.SMA(data["close"], timeperiod=50)
data["sma_200"] = talib.SMA(data["close"], timeperiod=200)

### Полосы Боллинджера (Bollinger Bands)

Полосы Боллинджера используются для измерения волатильности актива и выявления условий перекупленности и перепроданности. Полосы состоят из скользящей средней (обычно 20-дневной скользящей средней) и двух линий стандартного отклонения выше и ниже скользящей средней.

In [14]:
# Calculate the Bollinger Bands
data["upper_band"], data["middle_band"], data["lower_band"] = talib.BBANDS(data["close"], timeperiod=20)

### Индекс относительной силы (Relative Strength Index - RSI)

RSI - это индикатор импульса, который сравнивает величину недавних достижений с недавними потерями, чтобы определить условия перекупленности и перепроданности. Обычно он используется с 14-дневным периодом.

In [15]:
# Calculate the relative strength index (RSI)
data["RSI"] = talib.RSI(data["close"], timeperiod=14)

### Дивергенция сходимости скользящих средних (Moving Average Convergence Divergence - MACD)

MACD - это индикатор следования за трендом, который измеряет разницу между краткосрочной скользящей средней и долгосрочной скользящей средней. Обычно он используется с 12-дневной и 26-дневной скользящей средней, а также с 9-дневной сигнальной линией.

In [16]:
# Calculate the MACD
data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data["close"], fastperiod=12, slowperiod=26, signalperiod=9)

### Стохастический осциллятор (Stochastic Oscillator)

Стохастический осциллятор - это индикатор импульса, который сравнивает цену закрытия актива с его ценовым диапазоном за определенный период. Обычно он используется с 14-дневным периодом.

In [17]:
# Calculate the stochastic oscillator
data["stochastic_k"], data["stochastic_d"] = talib.STOCH(data["high"], data["low"], data["close"], fastk_period=14, slowk_period=3, slowd_period=3)